In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency


# Cleaning and some statistcs of combined data

In [ ]:
 df = pd.read_csv('clean RE and raw with nulls.csv')

<ipython-input-2-83254c1cecbf>:1: DtypeWarning: Columns (3,4,6,7,8,12,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('clean RE and raw with nulls.csv')


In [ ]:
df.columns

Index(['First Name', 'Last Name', 'Phone Area Code', 'Phone Number',
       'Lead Status', 'score', 'Lead Quality', 'Lead Source', 'Campaign Name',
       'Property Street Address', 'Property City', 'Property State',
       'Property Zip', 'Mailing Street Address', 'Mailing City',
       'Mailing State', 'Mailing Zip', 'Bedroom', 'Bathroom', 'Apporx Sqft',
       'Lot Size Sqft', 'Effective Year Built', 'House Type',
       'Tax Assessed Value', 'Last Sold Price', 'Lead Created Date',
       'download_date', 'same_address'],
      dtype='object')

In [ ]:
#remove all values where bathroom is greater than 54 to remove outliers
df.loc[df['Bathroom'] > 54, 'Bathroom'] = np.nan

In [ ]:
#remove all values where bedroom is greater than 65
df.loc[df['Bedroom'] > 65, 'Bedroom'] = np.nan

In [ ]:
#rename cities so no duplicates with differents spacing

In [ ]:
!pip install pandas fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Function to find similar city names and standardize them
def standardize_city_names(df, column, threshold=90):
    unique_cities = df[column].unique()

    # Ensure all elements are strings
    unique_cities = [str(city) for city in unique_cities]

    # Dictionary to map similar city names to a representative name
    city_mapping = {}

    for city in unique_cities:
        matches = process.extract(city, unique_cities, scorer=fuzz.token_sort_ratio)
        representative = city  # Choose the first encountered name as the representative
        for match in matches:
            if match[1] >= threshold:
                city_mapping[match[0]] = representative

    # Update the DataFrame with standardized city names
    df[column] = df[column].map(city_mapping)

    return df

In [ ]:
# Standardize city names
df = standardize_city_names(df,'Property City', threshold=90)
df = standardize_city_names(df,'Mailing City', threshold=90)

In [ ]:
df['Property City'].unique()

array([' Teaneck', 'Hollywood', 'Deerfield Beach', 'Boynton Beach',
       'Fort Lauderdale', 'Wilton Manors', 'Pompano Beach',
       'Delray Beach', ' Tampa', 'Plantation', 'West Palm Beach',
       ' Davenport', ' Apollo Beach', 'New Port Richey', ' Brandon',
       'Sunrise', ' Langhorne', 'Boca Raton', ' Cape Coral',
       'Pembroke Pines', 'Lauderdale Lakes', ' Riverview', ' Stuart',
       'Palm Beach Gardens', ' Miami Gardens', ' Miami', ' Palmetto Bay',
       'Davie', 'West Park', ' Port St. Lucie', 'Miramar',
       'Lake Worth Beach', 'Dania Beach', 'Greenacres', 'Coconut Creek',
       'Hallandale Beach', 'Tamarac', ' Aventura', ' Homestead',
       ' Miami Springs', ' Miami Beach', ' Ferndale', ' Winter Park',
       ' Apopka', ' Poinciana', ' Port Clinton', 'Land O Lakes',
       'Palm Beach', 'Lauderhill', ' Plant City', 'Oakland Park',
       ' Homosassa', ' Wimauma', 'Zephyrhills', 'Dade City',
       ' Winston-Salem', 'Coral Springs', ' Orlando', ' Dunnellon',
     

In [ ]:
df['Mailing City'].unique()

array(['Teaneck', 'Hollywood', nan, ..., 'Royal Oak', 'New Martinsville',
       'Chattanooga'], dtype=object)

In [ ]:
df['Property State'].unique()

array(['NJ', 'FL', 'PA', 'MI', 'OH', 'NC', nan, 'MA', 'NY', 'USA', 'MO',
       'CA', 'GA', 'IN'], dtype=object)

In [ ]:
#Create gender column from First name
!pip install gender-guesser
import gender_guesser.detector as gender

# Initialize the gender detector
d = gender.Detector()

# Function to predict gender based on name
def predict_gender(name):
  if not name or pd.isna(name):  # Check for empty or missing names
        return 'unknown'
  else:
        return d.get_gender(name)

# Apply the function to the 'First Name' column and create a new 'Gender' column
df['Gender'] = df['First Name'].apply(predict_gender)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.3/379.3 kB 8.8 MB/s eta 0:00:00


In [ ]:
df['Gender'].unique()

array(['male', 'female', 'unknown', 'mostly_female', 'mostly_male',
       'andy'], dtype=object)

In [ ]:
df['score'].unique()

array([ 1,  0,  3,  6, -1,  5,  2,  4,  7, -3])

In [ ]:
df['Lead at all'] = df['score'].apply(lambda x: 1 if x >= -1 else 0 if x < -1 else 'NA')

In [ ]:
columns_to_fill = ['Phone Area Code', 'Phone Number','Lead Status', 'Lead Quality', 'Lead Source','Campaign Name',
                   'Lead Created Date', 'download_date']

# Fill null values with 'None' in specified columns
df[columns_to_fill] = df[columns_to_fill].fillna('None')

In [ ]:
#save df to computer'
df.to_csv('Clean RE and raw with nullsX.csv')


In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(df['Gender'], df['Lead at all'])
print("\nContingency Table:")
print(contingency_table)

# Perform the Chi-Squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"\nChi-Squared Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the p-value
alpha = 0.05
if p <= alpha:
    print("\nThere is a significant association between gender and lead quality (reject H0).")
else:
    print("\nThere is no significant association between gender and lead quality (fail to reject H0).")



Contingency Table:
Lead Quality   Bad  Good   None   Ok
Gender                              
andy             4     2   1067    2
female         215    31  31054   76
male           447    53  62310  152
mostly_female   20     3   3167   12
mostly_male     25     1   2292    6
unknown        136    12  28544   42

Chi-Squared Statistic: 47.3125457712263
p-value: 3.28068275527956e-05
Degrees of Freedom: 15

Expected Frequencies:
Lead Quality          Bad       Good          None          Ok
Gender                                                        
andy             7.021701   0.845589   1064.728587    2.404124
female         204.942216  24.680172  31076.208494   70.169118
male           411.256113  49.525530  62360.410479  140.807878
mostly_female   20.914870   2.518674   3171.405520    7.160936
mostly_male     15.179937   1.828044   2301.794637    5.197381
unknown        187.685162  22.601991  28459.452284   64.260563

There is a significant association between gender and lead qua

In [ ]:
df_expected = pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns)

# Plot the observed frequencies
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.heatmap(contingency_table, annot=True, fmt="d", cmap="YlGnBu")
plt.title("Observed Frequencies")
plt.xlabel("Gender")
plt.ylabel("Lead at all")

# Plot the expected frequencies
plt.subplot(1, 2, 2)
sns.heatmap(df_expected, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title("Expected Frequencies")
plt.xlabel("Gender")
plt.ylabel("Lead at all")

# Display the plots
plt.tight_layout()
plt.show()

# Print chi-square test results
print(f"Chi2: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of Freedom: {dof}")

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [ ]:
df = pd.read_csv('both clean 2.csv')

<ipython-input-2-4534fa22e4f9>:1: DtypeWarning: Columns (4,5,7,8,9,13,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('both clean 2.csv')


In [ ]:
df.columns

Index(['Unnamed: 0', 'First Name', 'Last Name', 'Phone Area Code',
       'Phone Number', 'Lead Status', 'score', 'Lead Quality', 'Lead Source',
       'Campaign Name', 'Property Street Address', 'Property City',
       'Property State', 'Property Zip', 'Mailing Street Address',
       'Mailing City', 'Mailing State', 'Mailing Zip', 'Bedroom', 'Bathroom',
       'Apporx Sqft', 'Lot Size Sqft', 'Effective Year Built', 'House Type',
       'Tax Assessed Value', 'Last Sold Price', 'Lead Created Date',
       'download_date', 'same_address', 'Gender', 'Lead at all'],
      dtype='object')

In [ ]:
mean_tax_lead = df[df['Lead at all'] == 1]['Tax Assessed Value'].mean()
mean_tax_nolead = df[df['Lead at all'] == 0]['Tax Assessed Value'].mean()

print(f"Mean tax assessed value for leads: {mean_tax_lead:.2f}")
print(f"Mean tax assessed value for non-leads: {mean_tax_nolead:.2f}")

Mean tax assessed value for leads: 255017.87
Mean tax assessed value for non-leads: 131073.74


In [ ]:
mean_sold_lead = df[df['Lead at all'] == 1]['Last Sold Price'].mean()
mean_sold_nolead = df[df['Lead at all'] == 0]['Last Sold Price'].mean()

print(f"Mean last sold value for leads: {mean_sold_lead:.2f}")
print(f"Mean last sold value for non-leads: {mean_sold_nolead:.2f}")

Mean last sold value for leads: 225114.33
Mean last sold value for non-leads: 481504.74


In [ ]:
# Create a contingency table
contingency_table = pd.crosstab(df['House Type'], df['Lead at all'])
print("\nContingency Table:")
print(contingency_table)

# Perform the Chi-Squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print(f"\nChi-Squared Statistic: {chi2}")
print(f"p-value: {p}")
print(f"Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(pd.DataFrame(expected, index=contingency_table.index, columns=contingency_table.columns))

# Interpret the p-value
alpha = 0.05
if p <= alpha:
    print("\nThere is a significant association between house type and lead quality (reject H0).")
else:
    print("\nThere is no significant association between house type and lead quality (fail to reject H0).")



Contingency Table:
Lead at all                                 0    1
House Type                                        
Commercial                                  0    5
Condominium (Residential)                9232    0
Duplex (2 units, any combination)        1701    1
Industrial                                  0    2
Multi-Unit Professional                     0    3
Multi-Unit/Multi-Family Residential      3986   95
Quadruplex (4 units, any combination)     195    0
Residential                                 1  747
Single Family Residential              112858    5
Townhouse (Residential)                   198    0
Triplex (3 units, any combination)        263    0
Vacant Land                                 0    9

Chi-Squared Statistic: 114318.78736944296
p-value: 0.0
Degrees of Freedom: 11

Expected Frequencies:
Lead at all                                        0           1
House Type                                                      
Commercial                        